### Import Libraries & Dependencies

In [1]:
import scipy.io
import numpy as np
import numpy.linalg as la
import cvxopt.solvers
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import itertools
import collections
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import operator

### Import Data

In [2]:
mat = scipy.io.loadmat('MNIST_data.mat')
train_samples=mat['train_samples']
train_samples_labels=mat['train_samples_labels']
test_samples=mat['test_samples']
test_samples_labels=mat['test_samples_labels']

### Kernel

In [3]:
class Kernel(object):
   
    @staticmethod
    def gaussian(sigma):
        def f(x, y):
            exponent = -np.sqrt(la.norm(x-y) ** 2 / (2 * sigma ** 2))
            return np.exp(exponent)
        return f

    # For degree-d polynomials, the polynomial kernel is defined as
    #K(x,y)=(x^Ty+c)^{d}
    # where x and y are vectors in the input space
        #i.e. vectors of features computed from training or test samples
    #and c ≥ 0 is a free parameter trading off the influence of higher-order versus lower-order terms in the polynomial.
    #When c = 0, the kernel is called homogeneous.

    @staticmethod
    def polykernel(dimension):
        def f(x, y):
            return np.dot(x, y)** dimension
        return f


### SVM

In [4]:
MIN_SUPPORT_VECTOR_MULTIPLIER = 1e-5


class SVMTrainer(object):
    def __init__(self, kernel, c):
        self._kernel = kernel
        self._c = c

    def train(self, X, y):
        lagrange_multipliers = self._compute_multipliers(X, y)
        return self._construct_predictor(X, y, lagrange_multipliers)

    def _gram_matrix(self, X):
        n_samples, n_features = X.shape
        K = np.zeros((n_samples, n_samples))
        
        for i, x_i in enumerate(X):
            for j, x_j in enumerate(X):
                K[i, j] = self._kernel(x_i, x_j)
        return K

    def _construct_predictor(self, X, y, lagrange_multipliers):
        support_vector_indices = \
            lagrange_multipliers > MIN_SUPPORT_VECTOR_MULTIPLIER

        support_multipliers = lagrange_multipliers[support_vector_indices]
        support_vectors = X[support_vector_indices]
        support_vector_labels = y[support_vector_indices]

        # bias = y_k - \sum z_i y_i  K(x_k, x_i)
        bias = np.mean(
            [y_k - SVMPredictor(
                kernel=self._kernel,
                bias=0.0,
                weights=support_multipliers,
                support_vectors=support_vectors,
                support_vector_labels=support_vector_labels).predict(x_k)
             for (y_k, x_k) in zip(support_vector_labels, support_vectors)])

        return SVMPredictor(
            kernel=self._kernel,
            bias=bias,
            weights=support_multipliers,
            support_vectors=support_vectors,
            support_vector_labels=support_vector_labels)

    def _compute_multipliers(self, X, y):
        n_samples, n_features = X.shape

        K = self._gram_matrix(X)
        # Solves
        # min 1/2 x^T P x + q^T x
        # s.t.
        #  Gx \coneleq h
        #  Ax = b

        P = cvxopt.matrix(np.outer(y, y) * K)
        q = cvxopt.matrix(-1 * np.ones(n_samples))

        G_std = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        h_std = cvxopt.matrix(np.zeros(n_samples))

        G_slack = cvxopt.matrix(np.diag(np.ones(n_samples)))
        h_slack = cvxopt.matrix(np.ones(n_samples) * self._c)

        G = cvxopt.matrix(np.vstack((G_std, G_slack)))
        h = cvxopt.matrix(np.vstack((h_std, h_slack)))

        y = y.astype(np.double)
        A = cvxopt.matrix(y, (1, n_samples))
        b = cvxopt.matrix(0.0)

        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        return np.ravel(solution['x'])


class SVMPredictor(object):
    def __init__(self,
                 kernel,
                 bias,
                 weights,
                 support_vectors,
                 support_vector_labels):
        self._kernel = kernel
        self._bias = bias
        self._weights = weights
        self._support_vectors = support_vectors
        self._support_vector_labels = support_vector_labels
        assert len(support_vectors) == len(support_vector_labels)
        assert len(weights) == len(support_vector_labels)

    def predict(self, x):
        result = self._bias
        for z_i, x_i, y_i in zip(self._weights,
                                 self._support_vectors,
                                 self._support_vector_labels):
            result += z_i * y_i * self._kernel(x_i, x)
        return result

### One VS Rest

In [5]:
def OVR(X_train, Y_train, X_test, Y_test,sigma,C):   
    predictions=[]
    for i in range(10):
        temp_labels=np.copy(Y_train)
        temp_labels=np.where(temp_labels!=i, -1,temp_labels)
        temp_labels=np.where(temp_labels==i, 1,temp_labels)
        print(collections.Counter(temp_labels.ravel()))
        trainer = SVMTrainer(Kernel.gaussian(sigma), C)
        predictor = trainer.train(X_train, temp_labels)
        result=[]
        for dp in range(len(X_test)):
            p=predictor.predict(X_test[dp])
            result.append(p)
        predictions.append(result)
    predicted_labels=[]
    for dp in range(len(X_test)):
        a=[predictions[i][dp] for i in range(10)]
        predicted_labels.append(np.argmax(a))
    correct=0
    for i in range(len(X_test)):
        if predicted_labels[i]==Y_test[i]:
            correct+=1
    return predicted_labels, correct/1000

In [6]:
#ONE VS REST ON MNIST DATASET WITH GAUSSIAN KERNEL WITH SIGMA=0.5 and C= 1000
labels, accuracy =OVR(train_samples,train_samples_labels,test_samples,test_samples_labels,0.5,1000)
results = confusion_matrix(test_samples_labels,labels )
print(accuracy)
print(results)

Counter({-1: 3618, 1: 382})
     pcost       dcost       gap    pres   dres
 0:  3.4572e+07 -8.4541e+08  2e+09  2e-01  1e-12
 1:  2.4333e+07 -1.0495e+08  2e+08  2e-02  3e-12
 2:  9.1032e+06 -2.3282e+07  4e+07  3e-03  1e-12
 3:  2.0780e+06 -4.2657e+06  6e+06  2e-12  7e-13
 4:  3.0879e+05 -4.0792e+05  7e+05  2e-12  5e-13
 5:  4.3485e+04 -5.3217e+04  1e+05  1e-12  2e-13
 6:  5.7991e+03 -7.9232e+03  1e+04  5e-13  7e-14
 7:  6.2918e+02 -1.3147e+03  2e+03  8e-14  3e-14
 8: -3.9346e+00 -3.3583e+02  3e+02  4e-14  9e-15
 9: -8.3027e+01 -2.0139e+02  1e+02  9e-14  5e-15
10: -1.0988e+02 -1.5903e+02  5e+01  4e-14  4e-15
11: -1.2121e+02 -1.4142e+02  2e+01  5e-15  3e-15
12: -1.2637e+02 -1.3299e+02  7e+00  5e-15  4e-15
13: -1.2829e+02 -1.2928e+02  1e+00  4e-14  4e-15
14: -1.2865e+02 -1.2872e+02  7e-02  4e-14  4e-15
15: -1.2868e+02 -1.2868e+02  2e-03  5e-14  4e-15
16: -1.2868e+02 -1.2868e+02  3e-05  2e-15  4e-15
Optimal solution found.
Counter({-1: 3549, 1: 451})
     pcost       dcost       gap    pre

 9: -2.8323e+02 -4.7674e+02  2e+02  5e-15  7e-15
10: -3.2258e+02 -3.9486e+02  7e+01  1e-14  6e-15
11: -3.3828e+02 -3.6212e+02  2e+01  2e-14  5e-15
12: -3.4447e+02 -3.5057e+02  6e+00  4e-14  6e-15
13: -3.4612e+02 -3.4701e+02  9e-01  2e-14  6e-15
14: -3.4642e+02 -3.4645e+02  3e-02  9e-14  6e-15
15: -3.4643e+02 -3.4643e+02  7e-04  2e-14  6e-15
16: -3.4643e+02 -3.4643e+02  2e-05  1e-14  6e-15
Optimal solution found.
Counter({-1: 3599, 1: 401})
     pcost       dcost       gap    pres   dres
 0:  5.6478e+07 -8.0854e+08  1e+09  2e-01  2e-12
 1:  3.5781e+07 -1.1115e+08  2e+08  2e-02  4e-12
 2:  1.3977e+07 -3.5809e+07  6e+07  4e-03  1e-12
 3:  5.8588e+06 -1.6390e+07  2e+07  1e-03  7e-13
 4:  1.4703e+06 -4.2381e+06  6e+06  5e-13  5e-13
 5:  2.2980e+05 -3.6271e+05  6e+05  8e-13  3e-13
 6:  3.3283e+04 -4.5783e+04  8e+04  3e-14  1e-13
 7:  3.9775e+03 -6.9647e+03  1e+04  4e-13  5e-14
 8:  1.4918e+02 -1.3655e+03  2e+03  6e-14  2e-14
 9: -2.4036e+02 -6.2276e+02  4e+02  1e-14  9e-15
10: -3.1440e+02 -4

In [7]:
target_names = ['0', '1', '2','3','4','5','6','7','8','9']
print(classification_report(test_samples_labels,labels, target_names=target_names))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97        86
           1       0.98      0.99      0.99       122
           2       0.95      0.92      0.93       113
           3       0.94      0.89      0.91       115
           4       0.94      0.94      0.94       108
           5       0.92      0.90      0.91        92
           6       0.95      0.94      0.95        87
           7       0.93      0.94      0.93        99
           8       0.84      0.88      0.86        86
           9       0.93      0.97      0.95        92

   micro avg       0.94      0.94      0.94      1000
   macro avg       0.94      0.94      0.94      1000
weighted avg       0.94      0.94      0.94      1000



### One VS One

In [8]:
def OVO(X_train, Y_train, X_test, Y_test,sigma,C):

    overall_votes=[]
    for i in range(len(X_test)):
        a = range(10)
        d = dict.fromkeys(a, 0)
        overall_votes.append(d)
        
    for p in range(10):
        for n in range(p+1,10):
            positive= np.where(Y_train==p)[0]
            negative = np.where(Y_train==n)[0]
            positive_data= np.copy(X_train)[positive]
            negative_data = np.copy(X_train)[negative]
            data=np.vstack((positive_data,negative_data))
            labels=np.copy(Y_train)[0:(len(positive_data)+len(negative_data))]
            labels[0:len(positive_data)]=1
            labels=np.where(labels!=1, -1,labels)
            trainer = SVMTrainer(Kernel.gaussian(sigma), C)
            predictor = trainer.train(data,labels)
            for dp in range(len(X_test)):
                result=predictor.predict(X_test[dp])
                if result >= 0:
                    overall_votes[dp][p]+=1
                else:
                    overall_votes[dp][n]+=1
    labels=[]
    for dp in range(len(X_test)):
        label=max(overall_votes[dp].items(),key=operator.itemgetter(1))[0]
        labels.append(label)
    correct=0
    for i in range(len(X_test)):
        if labels[i]==Y_test[i]:
            correct+=1
    return labels, correct/1000

In [9]:
#ONE VS ONE ON MNIST DATASET WITH GAUSSIAN KERNEL WITH SIGMA=0.5 and C= 1000
labels,accuracy=OVO(train_samples,train_samples_labels,test_samples,test_samples_labels,0.5,1000)
results = confusion_matrix(test_samples_labels,labels )
print(accuracy)
print(results)

     pcost       dcost       gap    pres   dres
 0:  1.4908e+07 -1.7876e+08  3e+08  1e-01  1e-12
 1:  9.1858e+06 -2.9527e+07  4e+07  2e-02  2e-12
 2:  3.4035e+06 -9.2177e+06  1e+07  4e-03  1e-12
 3:  7.6079e+05 -1.9621e+06  3e+06  6e-12  6e-13
 4:  1.1948e+05 -1.7473e+05  3e+05  9e-13  3e-13
 5:  1.6106e+04 -2.1943e+04  4e+04  6e-13  1e-13
 6:  1.8061e+03 -3.5905e+03  5e+03  2e-13  4e-14
 7: -1.7263e+01 -7.5465e+02  7e+02  1e-13  2e-14
 8: -1.9818e+02 -3.3528e+02  1e+02  3e-14  8e-15
 9: -2.2366e+02 -2.5283e+02  3e+01  2e-14  5e-15
10: -2.2836e+02 -2.3909e+02  1e+01  2e-14  4e-15
11: -2.3021e+02 -2.3389e+02  4e+00  3e-14  4e-15
12: -2.3091e+02 -2.3158e+02  7e-01  4e-15  5e-15
13: -2.3107e+02 -2.3116e+02  9e-02  9e-16  5e-15
14: -2.3109e+02 -2.3110e+02  5e-03  1e-14  5e-15
15: -2.3109e+02 -2.3110e+02  1e-04  2e-14  5e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.1867e+07 -2.1612e+08  3e+08  1e-01  3e-12
 1:  1.1938e+07 -2.8419e+07  4e+07  1e-02  4e-1

     pcost       dcost       gap    pres   dres
 0:  1.9334e+07 -1.7138e+08  2e+08  9e-02  2e-12
 1:  9.8672e+06 -2.7813e+07  4e+07  1e-02  3e-12
 2:  2.7318e+06 -6.6179e+06  1e+07  1e-03  2e-12
 3:  4.5680e+05 -7.0509e+05  1e+06  8e-07  9e-13
 4:  6.4916e+04 -8.1748e+04  1e+05  1e-09  3e-13
 5:  8.5483e+03 -1.2174e+04  2e+04  5e-13  1e-13
 6:  8.6745e+02 -2.0613e+03  3e+03  1e-13  6e-14
 7: -7.6937e+01 -4.7124e+02  4e+02  4e-14  2e-14
 8: -1.6362e+02 -2.1997e+02  6e+01  3e-14  1e-14
 9: -1.7151e+02 -1.8905e+02  2e+01  5e-14  8e-15
10: -1.7413e+02 -1.7979e+02  6e+00  1e-14  7e-15
11: -1.7508e+02 -1.7668e+02  2e+00  3e-14  7e-15
12: -1.7542e+02 -1.7574e+02  3e-01  6e-14  8e-15
13: -1.7550e+02 -1.7552e+02  2e-02  5e-14  8e-15
14: -1.7551e+02 -1.7551e+02  5e-04  7e-15  8e-15
15: -1.7551e+02 -1.7551e+02  1e-05  4e-14  8e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.8156e+07 -1.5113e+08  2e+08  6e-02  2e-12
 1:  8.9342e+06 -2.5629e+07  4e+07  9e-03  2e-1

14: -1.7752e+02 -1.7752e+02  6e-05  2e-14  9e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.5282e+07 -2.3431e+08  4e+08  1e-01  4e-12
 1:  1.2768e+07 -2.8153e+07  4e+07  1e-02  5e-12
 2:  4.0856e+06 -9.0872e+06  1e+07  3e-03  3e-12
 3:  7.4645e+05 -1.4627e+06  2e+06  1e-11  2e-12
 4:  1.1185e+05 -1.4794e+05  3e+05  5e-12  7e-13
 5:  1.4933e+04 -2.0715e+04  4e+04  1e-12  3e-13
 6:  1.6174e+03 -3.4320e+03  5e+03  3e-13  1e-13
 7: -5.0932e+01 -7.4008e+02  7e+02  3e-13  5e-14
 8: -2.1270e+02 -2.9339e+02  8e+01  6e-14  2e-14
 9: -2.2452e+02 -2.4216e+02  2e+01  6e-15  2e-14
10: -2.2753e+02 -2.3149e+02  4e+00  4e-14  1e-14
11: -2.2844e+02 -2.2896e+02  5e-01  2e-14  1e-14
12: -2.2861e+02 -2.2864e+02  3e-02  2e-14  1e-14
13: -2.2862e+02 -2.2862e+02  9e-04  1e-14  1e-14
14: -2.2862e+02 -2.2862e+02  3e-05  3e-14  1e-14
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.2287e+07 -2.2098e+08  3e+08  1e-01  3e-12
 1:  1.2024e+07 -2.8921

     pcost       dcost       gap    pres   dres
 0:  2.4615e+07 -2.3814e+08  4e+08  2e-01  3e-12
 1:  1.3391e+07 -2.7651e+07  5e+07  1e-02  4e-12
 2:  4.4859e+06 -9.2098e+06  1e+07  3e-03  2e-12
 3:  8.2517e+05 -1.6180e+06  2e+06  6e-12  1e-12
 4:  1.2500e+05 -1.6485e+05  3e+05  2e-12  5e-13
 5:  1.6985e+04 -2.2524e+04  4e+04  1e-13  2e-13
 6:  1.9924e+03 -3.6186e+03  6e+03  2e-13  9e-14
 7:  5.7129e+01 -7.2075e+02  8e+02  1e-13  3e-14
 8: -1.4841e+02 -2.3975e+02  9e+01  4e-14  2e-14
 9: -1.6308e+02 -1.8318e+02  2e+01  1e-14  9e-15
10: -1.6632e+02 -1.7167e+02  5e+00  2e-14  8e-15
11: -1.6741e+02 -1.6836e+02  1e+00  2e-14  8e-15
12: -1.6767e+02 -1.6777e+02  1e-01  2e-14  8e-15
13: -1.6771e+02 -1.6771e+02  4e-03  2e-14  8e-15
14: -1.6771e+02 -1.6771e+02  1e-04  4e-15  9e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.2954e+07 -1.9822e+08  3e+08  1e-01  3e-12
 1:  1.1050e+07 -2.6818e+07  4e+07  1e-02  4e-12
 2:  3.6157e+06 -8.6623e+06  1e+07  3e-03  2e-1

12: -1.8739e+02 -1.8747e+02  8e-02  1e-13  9e-15
13: -1.8742e+02 -1.8742e+02  2e-03  6e-14  1e-14
14: -1.8742e+02 -1.8742e+02  4e-05  2e-16  1e-14
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.1151e+07 -1.9992e+08  3e+08  1e-01  3e-12
 1:  1.1000e+07 -2.7578e+07  4e+07  1e-02  3e-12
 2:  3.7244e+06 -9.1238e+06  1e+07  3e-03  2e-12
 3:  7.0553e+05 -1.4177e+06  2e+06  6e-13  1e-12
 4:  1.0725e+05 -1.4429e+05  3e+05  1e-12  4e-13
 5:  1.4408e+04 -1.9652e+04  3e+04  5e-13  2e-13
 6:  1.6059e+03 -3.2253e+03  5e+03  4e-13  7e-14
 7: -1.6745e+01 -6.7917e+02  7e+02  3e-14  3e-14
 8: -1.8044e+02 -2.5849e+02  8e+01  4e-15  1e-14
 9: -1.9288e+02 -2.0961e+02  2e+01  1e-14  8e-15
10: -1.9585e+02 -1.9985e+02  4e+00  2e-14  7e-15
11: -1.9671e+02 -1.9752e+02  8e-01  4e-15  7e-15
12: -1.9695e+02 -1.9703e+02  9e-02  7e-15  8e-15
13: -1.9697e+02 -1.9698e+02  3e-03  2e-14  7e-15
14: -1.9698e+02 -1.9698e+02  9e-05  4e-15  7e-15
Optimal solution found.
     pcost       dcost

In [10]:
target_names = ['0', '1', '2','3','4','5','6','7','8','9']
print(classification_report(test_samples_labels,labels, target_names=target_names))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96        86
           1       0.96      1.00      0.98       122
           2       0.86      0.98      0.92       113
           3       0.92      0.90      0.91       115
           4       0.93      0.94      0.94       108
           5       0.89      0.90      0.90        92
           6       0.96      0.89      0.92        87
           7       0.92      0.91      0.91        99
           8       0.95      0.84      0.89        86
           9       0.98      0.91      0.94        92

   micro avg       0.93      0.93      0.93      1000
   macro avg       0.93      0.92      0.93      1000
weighted avg       0.93      0.93      0.93      1000



### DAGSVM

#### Gaussian Kernel DAGSVM

In [11]:
def gaussian_DAGSVM(X_train, Y_train, X_test, Y_test,sigma,C):

    svms=[]
    for dp in range(len(X_train)):
        svm_names=[]
        for i in range(10):
            for j in range(i+1,10):
                svm_name=str(i)+"_vs_"+str(j)
                svm_names.append(svm_name)
        d = dict.fromkeys(svm_names, 0)
        svms.append(d)
        
    for p in range(10):
        for n in range(p+1,10):
            positive= np.where(Y_train==p)[0]
            negative = np.where(Y_train==n)[0]
            positive_data= np.copy(X_train)[positive]
            negative_data = np.copy(X_train)[negative]
            data=np.vstack((positive_data,negative_data))
            labels=np.copy(Y_train)[0:(len(positive_data)+len(negative_data))]
            labels[0:len(positive_data)]=1
            labels=np.where(labels!=1, -1,labels)
            trainer = SVMTrainer(Kernel.gaussian(sigma), C)
            predictor = trainer.train(data,labels)
            for dp in range(len(X_test)):
                result=predictor.predict(X_test[dp])
                svm_name=str(p)+"_vs_"+str(n)
                svms[dp][svm_name]=result
    
    labels=[]
    for dp in range(len(X_test)):
        options=np.arange(10)
        p,n=0,9
        while len(options)>1:
            svm_name=str(p)+"_vs_"+str(n)
            if svms[dp][svm_name]>0:
                index=int(np.where(options==n)[0])
                options=np.delete(options,index)
                n-=1                
            else:
                index=int(np.where(options==p)[0])
                options=np.delete(options,index)
                p+=1
        labels.append(options)
        
    correct=0
    for i in range(len(X_test)):
        if labels[i]==Y_test[i]:
            correct+=1
    return labels, correct/1000

In [12]:
#DAGSVM ON MNIST DATASET WITH GAUSSIAN KERNEL WITH SIGMA=0.5 and C= 1000
labels,accuracy=gaussian_DAGSVM(train_samples,train_samples_labels,test_samples,test_samples_labels,0.5,1000)
results = confusion_matrix(test_samples_labels,labels )
print(accuracy)
print(results)

     pcost       dcost       gap    pres   dres
 0:  1.4908e+07 -1.7876e+08  3e+08  1e-01  1e-12
 1:  9.1858e+06 -2.9527e+07  4e+07  2e-02  2e-12
 2:  3.4035e+06 -9.2177e+06  1e+07  4e-03  1e-12
 3:  7.6079e+05 -1.9621e+06  3e+06  6e-12  6e-13
 4:  1.1948e+05 -1.7473e+05  3e+05  9e-13  3e-13
 5:  1.6106e+04 -2.1943e+04  4e+04  6e-13  1e-13
 6:  1.8061e+03 -3.5905e+03  5e+03  2e-13  4e-14
 7: -1.7263e+01 -7.5465e+02  7e+02  1e-13  2e-14
 8: -1.9818e+02 -3.3528e+02  1e+02  3e-14  8e-15
 9: -2.2366e+02 -2.5283e+02  3e+01  2e-14  5e-15
10: -2.2836e+02 -2.3909e+02  1e+01  2e-14  4e-15
11: -2.3021e+02 -2.3389e+02  4e+00  3e-14  4e-15
12: -2.3091e+02 -2.3158e+02  7e-01  4e-15  5e-15
13: -2.3107e+02 -2.3116e+02  9e-02  9e-16  5e-15
14: -2.3109e+02 -2.3110e+02  5e-03  1e-14  5e-15
15: -2.3109e+02 -2.3110e+02  1e-04  2e-14  5e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.1867e+07 -2.1612e+08  3e+08  1e-01  3e-12
 1:  1.1938e+07 -2.8419e+07  4e+07  1e-02  4e-1

     pcost       dcost       gap    pres   dres
 0:  1.9334e+07 -1.7138e+08  2e+08  9e-02  2e-12
 1:  9.8672e+06 -2.7813e+07  4e+07  1e-02  3e-12
 2:  2.7318e+06 -6.6179e+06  1e+07  1e-03  2e-12
 3:  4.5680e+05 -7.0509e+05  1e+06  8e-07  9e-13
 4:  6.4916e+04 -8.1748e+04  1e+05  1e-09  3e-13
 5:  8.5483e+03 -1.2174e+04  2e+04  5e-13  1e-13
 6:  8.6745e+02 -2.0613e+03  3e+03  1e-13  6e-14
 7: -7.6937e+01 -4.7124e+02  4e+02  4e-14  2e-14
 8: -1.6362e+02 -2.1997e+02  6e+01  3e-14  1e-14
 9: -1.7151e+02 -1.8905e+02  2e+01  5e-14  8e-15
10: -1.7413e+02 -1.7979e+02  6e+00  1e-14  7e-15
11: -1.7508e+02 -1.7668e+02  2e+00  3e-14  7e-15
12: -1.7542e+02 -1.7574e+02  3e-01  6e-14  8e-15
13: -1.7550e+02 -1.7552e+02  2e-02  5e-14  8e-15
14: -1.7551e+02 -1.7551e+02  5e-04  7e-15  8e-15
15: -1.7551e+02 -1.7551e+02  1e-05  4e-14  8e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.8156e+07 -1.5113e+08  2e+08  6e-02  2e-12
 1:  8.9342e+06 -2.5629e+07  4e+07  9e-03  2e-1

13: -1.7752e+02 -1.7752e+02  2e-03  9e-14  9e-15
14: -1.7752e+02 -1.7752e+02  6e-05  2e-14  9e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.5282e+07 -2.3431e+08  4e+08  1e-01  4e-12
 1:  1.2768e+07 -2.8153e+07  4e+07  1e-02  5e-12
 2:  4.0856e+06 -9.0872e+06  1e+07  3e-03  3e-12
 3:  7.4645e+05 -1.4627e+06  2e+06  1e-11  2e-12
 4:  1.1185e+05 -1.4794e+05  3e+05  5e-12  7e-13
 5:  1.4933e+04 -2.0715e+04  4e+04  1e-12  3e-13
 6:  1.6174e+03 -3.4320e+03  5e+03  3e-13  1e-13
 7: -5.0932e+01 -7.4008e+02  7e+02  3e-13  5e-14
 8: -2.1270e+02 -2.9339e+02  8e+01  6e-14  2e-14
 9: -2.2452e+02 -2.4216e+02  2e+01  6e-15  2e-14
10: -2.2753e+02 -2.3149e+02  4e+00  4e-14  1e-14
11: -2.2844e+02 -2.2896e+02  5e-01  2e-14  1e-14
12: -2.2861e+02 -2.2864e+02  3e-02  2e-14  1e-14
13: -2.2862e+02 -2.2862e+02  9e-04  1e-14  1e-14
14: -2.2862e+02 -2.2862e+02  3e-05  3e-14  1e-14
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.2287e+07 -2.2098

     pcost       dcost       gap    pres   dres
 0:  2.4615e+07 -2.3814e+08  4e+08  2e-01  3e-12
 1:  1.3391e+07 -2.7651e+07  5e+07  1e-02  4e-12
 2:  4.4859e+06 -9.2098e+06  1e+07  3e-03  2e-12
 3:  8.2517e+05 -1.6180e+06  2e+06  6e-12  1e-12
 4:  1.2500e+05 -1.6485e+05  3e+05  2e-12  5e-13
 5:  1.6985e+04 -2.2524e+04  4e+04  1e-13  2e-13
 6:  1.9924e+03 -3.6186e+03  6e+03  2e-13  9e-14
 7:  5.7129e+01 -7.2075e+02  8e+02  1e-13  3e-14
 8: -1.4841e+02 -2.3975e+02  9e+01  4e-14  2e-14
 9: -1.6308e+02 -1.8318e+02  2e+01  1e-14  9e-15
10: -1.6632e+02 -1.7167e+02  5e+00  2e-14  8e-15
11: -1.6741e+02 -1.6836e+02  1e+00  2e-14  8e-15
12: -1.6767e+02 -1.6777e+02  1e-01  2e-14  8e-15
13: -1.6771e+02 -1.6771e+02  4e-03  2e-14  8e-15
14: -1.6771e+02 -1.6771e+02  1e-04  4e-15  9e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.2954e+07 -1.9822e+08  3e+08  1e-01  3e-12
 1:  1.1050e+07 -2.6818e+07  4e+07  1e-02  4e-12
 2:  3.6157e+06 -8.6623e+06  1e+07  3e-03  2e-1

13: -1.8742e+02 -1.8742e+02  2e-03  6e-14  1e-14
14: -1.8742e+02 -1.8742e+02  4e-05  2e-16  1e-14
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.1151e+07 -1.9992e+08  3e+08  1e-01  3e-12
 1:  1.1000e+07 -2.7578e+07  4e+07  1e-02  3e-12
 2:  3.7244e+06 -9.1238e+06  1e+07  3e-03  2e-12
 3:  7.0553e+05 -1.4177e+06  2e+06  6e-13  1e-12
 4:  1.0725e+05 -1.4429e+05  3e+05  1e-12  4e-13
 5:  1.4408e+04 -1.9652e+04  3e+04  5e-13  2e-13
 6:  1.6059e+03 -3.2253e+03  5e+03  4e-13  7e-14
 7: -1.6745e+01 -6.7917e+02  7e+02  3e-14  3e-14
 8: -1.8044e+02 -2.5849e+02  8e+01  4e-15  1e-14
 9: -1.9288e+02 -2.0961e+02  2e+01  1e-14  8e-15
10: -1.9585e+02 -1.9985e+02  4e+00  2e-14  7e-15
11: -1.9671e+02 -1.9752e+02  8e-01  4e-15  7e-15
12: -1.9695e+02 -1.9703e+02  9e-02  7e-15  8e-15
13: -1.9697e+02 -1.9698e+02  3e-03  2e-14  7e-15
14: -1.9698e+02 -1.9698e+02  9e-05  4e-15  7e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.2340e+07 -1.9807

In [13]:
target_names = ['0', '1', '2','3','4','5','6','7','8','9']
print(classification_report(test_samples_labels,labels, target_names=target_names))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97        86
           1       0.97      1.00      0.98       122
           2       0.91      0.96      0.94       113
           3       0.91      0.91      0.91       115
           4       0.91      0.95      0.93       108
           5       0.89      0.91      0.90        92
           6       0.96      0.89      0.92        87
           7       0.90      0.90      0.90        99
           8       0.92      0.85      0.88        86
           9       0.98      0.90      0.94        92

   micro avg       0.93      0.93      0.93      1000
   macro avg       0.93      0.93      0.93      1000
weighted avg       0.93      0.93      0.93      1000



#### Polynomial Kernel DAGSVM

In [14]:
def DAGSVM(X_train, Y_train, X_test, Y_test,degree,C):

    svms=[]
    for dp in range(len(X_train)):
        svm_names=[]
        for i in range(10):
            for j in range(i+1,10):
                svm_name=str(i)+"_vs_"+str(j)
                svm_names.append(svm_name)
        d = dict.fromkeys(svm_names, 0)
        svms.append(d)
        
    for p in range(10):
        for n in range(p+1,10):
            positive= np.where(Y_train==p)[0]
            negative = np.where(Y_train==n)[0]
            positive_data= np.copy(X_train)[positive]
            negative_data = np.copy(X_train)[negative]
            data=np.vstack((positive_data,negative_data))
            labels=np.copy(Y_train)[0:(len(positive_data)+len(negative_data))]
            labels[0:len(positive_data)]=1
            labels=np.where(labels!=1, -1,labels)
            trainer = SVMTrainer(Kernel.polykernel(degree), C)
            predictor = trainer.train(data,labels)
            for dp in range(len(X_test)):
                result=predictor.predict(X_test[dp])
                svm_name=str(p)+"_vs_"+str(n)
                svms[dp][svm_name]=result
    
    labels=[]
    for dp in range(len(X_test)):
        options=np.arange(10)
        p,n=0,9
        while len(options)>1:
            svm_name=str(p)+"_vs_"+str(n)
            if svms[dp][svm_name]>0:
                index=int(np.where(options==n)[0])
                options=np.delete(options,index)
                n-=1                
            else:
                index=int(np.where(options==p)[0])
                options=np.delete(options,index)
                p+=1
        labels.append(options)
        
    correct=0
    for i in range(len(X_test)):
        if labels[i]==Y_test[i]:
            correct+=1
    return labels, correct/1000

In [15]:
labels,accuracy=DAGSVM(train_samples,train_samples_labels,test_samples,test_samples_labels,6,1)
results = confusion_matrix(test_samples_labels,labels )
print(accuracy)
print(results)

     pcost       dcost       gap    pres   dres
 0: -1.4024e+02 -1.7227e+03  8e+03  2e+00  1e-15
 1: -9.6005e+01 -1.0049e+03  1e+03  2e-01  1e-15
 2: -1.0101e+02 -2.2316e+02  1e+02  1e-02  1e-15
 3: -1.1326e+02 -1.4655e+02  3e+01  2e-03  1e-15
 4: -1.1778e+02 -1.2796e+02  1e+01  7e-05  1e-15
 5: -1.1954e+02 -1.2158e+02  2e+00  6e-06  1e-15
 6: -1.2000e+02 -1.2039e+02  4e-01  3e-14  1e-15
 7: -1.2010e+02 -1.2014e+02  5e-02  2e-14  1e-15
 8: -1.2011e+02 -1.2011e+02  1e-03  4e-15  1e-15
 9: -1.2011e+02 -1.2011e+02  4e-05  2e-14  1e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.3084e+02 -1.5856e+03  7e+03  2e+00  1e-15
 1: -1.0265e+02 -9.5402e+02  1e+03  2e-01  2e-15
 2: -1.1125e+02 -2.2270e+02  1e+02  1e-02  2e-15
 3: -1.2483e+02 -1.4964e+02  3e+01  2e-03  1e-15
 4: -1.2939e+02 -1.3385e+02  5e+00  3e-04  1e-15
 5: -1.3053e+02 -1.3116e+02  6e-01  3e-05  1e-15
 6: -1.3073e+02 -1.3078e+02  5e-02  2e-06  1e-15
 7: -1.3075e+02 -1.3075e+02  2e-03  4e-08  1e-1

     pcost       dcost       gap    pres   dres
 0: -1.2199e+02 -1.7114e+03  8e+03  2e+00  1e-15
 1: -8.7542e+01 -9.7057e+02  1e+03  1e-01  1e-15
 2: -9.8965e+01 -2.1526e+02  1e+02  1e-02  2e-15
 3: -1.0957e+02 -1.3631e+02  3e+01  2e-03  1e-15
 4: -1.1293e+02 -1.2077e+02  8e+00  4e-04  1e-15
 5: -1.1425e+02 -1.1609e+02  2e+00  4e-05  1e-15
 6: -1.1463e+02 -1.1502e+02  4e-01  5e-06  1e-15
 7: -1.1473e+02 -1.1475e+02  2e-02  8e-08  1e-15
 8: -1.1474e+02 -1.1474e+02  5e-04  1e-09  1e-15
 9: -1.1474e+02 -1.1474e+02  2e-05  3e-11  1e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.6949e+02 -1.9102e+03  9e+03  3e+00  2e-15
 1: -1.3243e+02 -1.2255e+03  1e+03  1e-01  2e-15
 2: -1.4928e+02 -2.9067e+02  1e+02  1e-02  2e-15
 3: -1.6438e+02 -1.9681e+02  3e+01  2e-03  2e-15
 4: -1.6935e+02 -1.7505e+02  6e+00  2e-04  2e-15
 5: -1.7056e+02 -1.7122e+02  7e-01  2e-05  2e-15
 6: -1.7074e+02 -1.7080e+02  6e-02  1e-06  2e-15
 7: -1.7076e+02 -1.7076e+02  2e-03  2e-08  2e-1

     pcost       dcost       gap    pres   dres
 0: -1.5732e+02 -1.7062e+03  7e+03  2e+00  2e-15
 1: -1.2090e+02 -1.0274e+03  1e+03  1e-01  2e-15
 2: -1.3771e+02 -2.6329e+02  1e+02  1e-02  2e-15
 3: -1.4968e+02 -1.7825e+02  3e+01  2e-03  1e-15
 4: -1.5371e+02 -1.5967e+02  6e+00  3e-04  1e-15
 5: -1.5475e+02 -1.5593e+02  1e+00  5e-05  1e-15
 6: -1.5502e+02 -1.5511e+02  9e-02  1e-06  1e-15
 7: -1.5504e+02 -1.5505e+02  3e-03  4e-08  1e-15
 8: -1.5505e+02 -1.5505e+02  9e-05  6e-10  2e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.3894e+02 -1.7018e+03  7e+03  2e+00  1e-15
 1: -1.0713e+02 -1.0349e+03  1e+03  1e-01  2e-15
 2: -1.2515e+02 -2.3749e+02  1e+02  9e-03  2e-15
 3: -1.3727e+02 -1.5915e+02  2e+01  1e-03  1e-15
 4: -1.4062e+02 -1.4512e+02  5e+00  2e-04  1e-15
 5: -1.4152e+02 -1.4228e+02  8e-01  2e-05  1e-15
 6: -1.4169e+02 -1.4175e+02  6e-02  1e-06  1e-15
 7: -1.4171e+02 -1.4171e+02  2e-03  3e-08  1e-15
 8: -1.4171e+02 -1.4171e+02  9e-05  4e-10  1e-1

In [16]:
target_names = ['0', '1', '2','3','4','5','6','7','8','9']
print(classification_report(test_samples_labels,labels, target_names=target_names))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97        86
           1       0.98      1.00      0.99       122
           2       0.83      0.98      0.90       113
           3       0.89      0.93      0.91       115
           4       0.85      0.94      0.89       108
           5       0.89      0.92      0.91        92
           6       0.99      0.85      0.91        87
           7       0.93      0.84      0.88        99
           8       0.96      0.79      0.87        86
           9       0.96      0.88      0.92        92

   micro avg       0.92      0.92      0.92      1000
   macro avg       0.92      0.91      0.92      1000
weighted avg       0.92      0.92      0.92      1000

